In [ ]:
# load dataset
import pandas as pd
datrain = pd.read_csv("train.csv")
datest= pd.read_csv("test.csv")

In [ ]:
datrain["y"] = datrain["y"].astype("category")
datrain["country"] = datrain["country"].astype("category")
datest["country"] = datest["country"].astype("category")


#datrain["difflevel"] = datrain["difflevel"].astype("category")
#datest["difflevel"] = datest["difflevel"].astype("category")

In [ ]:
from sklearn.model_selection import train_test_split

train_train, train_val = train_test_split(datrain, train_size=0.7, stratify=datrain['y'])

In [ ]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
# Dividir los datos en características (X) y variable objetivo (y)
X_train = train_train.drop(['y'], axis=1)
y_train = train_train['y']
X_val = train_val.drop(['y'], axis=1)
y_val = train_val['y']

# Convertir las variables categóricas en numéricas utilizando codificación one-hot
categorical_features = ['country']
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), categorical_features)], remainder='passthrough')
X_train = ct.fit_transform(X_train)
X_val = ct.transform(X_val)

# Normalizar los datos utilizando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)    
 

# boosting catboost

In [ ]:

import optuna
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

train_pool = Pool(data=X_train, label=y_train)
val_pool = Pool(data=X_val, label=y_val)

import optuna
from optuna.samplers import TPESampler

def objective(trial):
    # Suggérer des valeurs pour les hyperparamètres
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True)
    random_strength = trial.suggest_float('random_strength', 1e-8, 10.0, log=True)
    subsample = trial.suggest_float('subsample', 0.2, 1)
    bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS'])
    
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        random_strength=random_strength,
        subsample=subsample,
        bootstrap_type=bootstrap_type,
        loss_function='MultiClass',
        eval_metric='MultiClass',
        random_seed=1,
        verbose=False,  # silence training
        early_stopping_rounds=100,
        task_type='CPU'
    )
    
    model.fit(train_pool, eval_set=val_pool)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

sampler = TPESampler(seed=1)
study_cb = optuna.create_study(direction="maximize", sampler=sampler)
study_cb.optimize(objective, n_trials=200)

print("Number of finished trials: {}".format(len(study_cb.trials)))
print("Best trial:")
trial = study_cb.best_trial
print("  Value (accuracy): {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
# Récupérer les meilleurs hyperparamètres trouvés
best_params_CatBoost = study_cb.best_params

# Créer un modèle CatBoost avec ces hyperparamètres
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=best_params_CatBoost['learning_rate'],
    depth=best_params_CatBoost['depth'],
    l2_leaf_reg=best_params_CatBoost['l2_leaf_reg'],
    random_strength=best_params_CatBoost['random_strength'],
    subsample=best_params_CatBoost['subsample'],
    bootstrap_type=best_params_CatBoost['bootstrap_type'],
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=1,
    verbose=200,  # Afficher les logs chaque 200 itérations
    early_stopping_rounds=100,
    task_type='CPU'
)

# Entraîner le modèle avec les données d'entraînement
model.fit(train_pool, eval_set=val_pool)

# Prédire sur l'ensemble de validation
y_pred_CatBoost = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred_CatBoost)
print(f"Accuracy (Taux de bonne classification avec les meilleurs hyperparamètres): {accuracy * 100:.2f}%")

In [ ]:
best_params_CatBoost

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
# 1. Préparer l'ensemble complet de données d'entraînement (datrain) et les données de test (datest).

X_full_train = datrain.drop(['y'], axis=1)
y_full_train = datrain['y']

# 2. Convertisser les variables catégoriques en numériques en utilisant la codification one-hot.

# Ajuster le transformateur sur les données d'entraînement complètes et les transformer
X_full_train_encoded = ct.fit_transform(X_full_train)

datest_1 = datest.drop('id', axis=1)
# Transformer les données de test
X_test_encoded = ct.transform(datest_1)

# Créer des pools pour l'entraînement et le test
full_train_pool = Pool(data=X_full_train_encoded, label=y_full_train)
test_pool = Pool(data=X_test_encoded)

In [ ]:
# Entraîner le modèle avec les données d'entraînement
model_full = CatBoostClassifier(
    iterations=1000,
    learning_rate=best_params_CatBoost['learning_rate'],
    depth=best_params_CatBoost['depth'],
    l2_leaf_reg=best_params_CatBoost['l2_leaf_reg'],
    random_strength=best_params_CatBoost['random_strength'],
    subsample=best_params_CatBoost['subsample'],
    bootstrap_type=best_params_CatBoost['bootstrap_type'],
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=1,
    verbose=200,
    task_type='CPU'
)

model_full.fit(full_train_pool)



# soumission

In [ ]:
# prédictions dans la base de données datest
y_test_pred = model_full.predict(test_pool)
CatBoost_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_test_pred.flatten()})
CatBoost_model_soumission.dtypes

In [ ]:
CatBoost_model_soumission

In [ ]:

pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
CatBoost_model_soumission.to_csv(pa + "CatBoost_model_soumission.csv", index=False)


# boosting lightgbm

In [ ]:
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgb
from sklearn.metrics import accuracy_score

def objective(trial):
    # Suggérer des valeurs pour les hyperparamètres
    num_leaves = trial.suggest_int("num_leaves", 2, 650)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
    min_child_samples = trial.suggest_int("min_child_samples", 5, 100)
    feature_fraction = trial.suggest_float("feature_fraction", 0.4, 1.0)
    bagging_fraction = trial.suggest_float("bagging_fraction", 0.4, 1.0)
    bagging_freq = trial.suggest_int("bagging_freq", 1, 7)
    lambda_l1 = trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True)
    lambda_l2 = trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True)

    classifier = lgb.LGBMClassifier(
        boosting_type='gbdt',
        objective='multiclass',
        metric='multi_logloss',
        num_class=4,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        min_child_samples=min_child_samples,
        feature_fraction=feature_fraction,
        bagging_fraction=bagging_fraction,
        bagging_freq=bagging_freq,
        lambda_l1=lambda_l1,
        lambda_l2=lambda_l2,
        random_seed=1
    )

    classifier.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0, early_stopping_rounds=100)
    
    y_pred = classifier.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy


sampler = TPESampler(seed=1)
study_lg = optuna.create_study(direction="maximize", sampler=sampler)
study_lg.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study_lg.trials)))
print("Best trial:")
trial = study_lg.best_trial
print("  Value (accuracy): {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#best_params = trial.params
best_params_lg = study_lg.best_params

best_lgbm = lgb.LGBMClassifier(
    num_leaves=best_params_lg["num_leaves"],
    learning_rate=best_params_lg["learning_rate"],
    min_child_samples=best_params_lg["min_child_samples"],
    feature_fraction=best_params_lg["feature_fraction"],
    bagging_fraction=best_params_lg["bagging_fraction"],
    bagging_freq=best_params_lg["bagging_freq"],
    lambda_l1=best_params_lg["lambda_l1"],
    lambda_l2=best_params_lg["lambda_l2"],
    random_seed=1
)

best_lgbm.fit(X_train, y_train)

y_pred_val_lgbm = best_lgbm.predict(X_val)
accuracy_val_lgbm = accuracy_score(y_val, y_pred_val_lgbm)
print(f"Accuracy of LGBM on validation set: {accuracy_val_lgbm:.4f}")


In [ ]:
best_params_lg

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
model_full_lgbm = lgb.LGBMClassifier(
    num_leaves=best_params_lg["num_leaves"],
    learning_rate=best_params_lg["learning_rate"],
    min_child_samples=best_params_lg["min_child_samples"],
    feature_fraction=best_params_lg["feature_fraction"],
    bagging_fraction=best_params_lg["bagging_fraction"],
    bagging_freq=best_params_lg["bagging_freq"],
    lambda_l1=best_params_lg["lambda_l1"],
    lambda_l2=best_params_lg["lambda_l2"],
    random_seed=1
)

model_full_lgbm.fit(X_full_train_encoded, y_full_train)

# soumission

In [ ]:
# prédictions dans la base de données datest
y_test_pred = model_full_lgbm.predict(X_test_encoded)
Lgb_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_test_pred.flatten()})
Lgb_model_soumission.dtypes

In [ ]:
Lgb_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
Lgb_model_soumission.to_csv(pa + "Lgb_model_soumission.csv", index=False)


# boosting xgboost

In [ ]:
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Conversion des labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

def objective_xgb(trial):
    # Définition des hyperparamètres à optimiser
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1),
        "gamma": trial.suggest_float("gamma", 0, 1, step=0.1),
        "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
        "lambda_": trial.suggest_float("lambda_", 1e-8, 10.0, log=True)
    }

    xgb_classifier = xgb.XGBClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        learning_rate=params["learning_rate"],
        subsample=params["subsample"],
        colsample_bytree=params["colsample_bytree"],
        gamma=params["gamma"],
        reg_alpha=params["alpha"],
        reg_lambda=params["lambda_"],
        objective="multi:softmax",
        num_class=len(le.classes_),
        random_state=1
    )

    xgb_classifier.fit(X_train_scaled, y_train_encoded, eval_set=[(X_val_scaled, y_val_encoded)], verbose=False, early_stopping_rounds=50)
    
    y_pred_encoded = xgb_classifier.predict(X_val_scaled)
    y_pred = le.inverse_transform(y_pred_encoded)

    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

sampler_xgb = optuna.samplers.TPESampler(seed=1)
study_xgb = optuna.create_study(direction="maximize", sampler=sampler_xgb)
study_xgb.optimize(objective_xgb, n_trials=100)  # Ajustez n_trials selon les besoins

print("Number of finished trials: {}".format(len(study_xgb.trials)))
print("Best trial:")
trial = study_xgb.best_trial
print("Value: {}".format(trial.value))
print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



In [ ]:
#best_params = trial.params
best_params_xgb = study_xgb.best_params

best_xgb_classifier = xgb.XGBClassifier(
    n_estimators=best_params_xgb["n_estimators"],
    max_depth=best_params_xgb["max_depth"],
    learning_rate=best_params_xgb["learning_rate"],
    subsample=best_params_xgb["subsample"],
    colsample_bytree=best_params_xgb["colsample_bytree"],
    gamma=best_params_xgb["gamma"],
    reg_alpha=best_params_xgb["alpha"],
    reg_lambda=best_params_xgb["lambda_"],
    objective="multi:softmax",
    num_class=len(le.classes_),
    random_state=1
)

best_xgb_classifier.fit(X_train_scaled, y_train_encoded)


y_val_pred_encoded = best_xgb_classifier.predict(X_val_scaled)
y_val_pred = le.inverse_transform(y_val_pred_encoded)

accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of Best Model: {accuracy}")


In [ ]:
best_params_xgb

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
model_full_xgb = xgb.XGBClassifier(
    n_estimators=best_params_xgb["n_estimators"],
    max_depth=best_params_xgb["max_depth"],
    learning_rate=best_params_xgb["learning_rate"],
    subsample=best_params_xgb["subsample"],
    colsample_bytree=best_params_xgb["colsample_bytree"],
    gamma=best_params_xgb["gamma"],
    reg_alpha=best_params_xgb["alpha"],
    reg_lambda=best_params_xgb["lambda_"],
    objective="multi:softmax",
    num_class=len(le.classes_),
    random_state=1
)

# Conversion des labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_full_train)



model_full_xgb.fit(X_full_train_encoded, y_train_encoded)

#y_pred_encoded = xgb_classifier.predict(X_val_scaled)
   # y_pred = le.inverse_transform(y_pred_encoded)



# soumission

In [ ]:
# prédictions dans la base de données datest
y_pred_encoded = model_full_xgb.predict(X_test_encoded)
y_test_pred = le.inverse_transform(y_pred_encoded)
xgb_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_test_pred.flatten()})
xgb_model_soumission.dtypes

In [ ]:
xgb_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
xgb_model_soumission.to_csv(pa + "xgb_model_soumission.csv", index=False)


# GradientBoosting 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import optuna

# Conversion des labels (si nécessaire)
#le = LabelEncoder()
#y_train_encoded = le.fit_transform(y_train)
#y_val_encoded = le.transform(y_val)

def objective_gbc(trial):
    # Définition des hyperparamètres à optimiser
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
    }

    gbc = GradientBoostingClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        learning_rate=params["learning_rate"],
        subsample=params["subsample"],
        max_features=params["max_features"],
        min_samples_split=params["min_samples_split"],
        min_samples_leaf=params["min_samples_leaf"],
        random_state=1
    )

    gbc.fit(X_train_scaled, y_train)
    y_pred = gbc.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

sampler_gbc = optuna.samplers.TPESampler(seed=1)
study_gbc = optuna.create_study(direction="maximize", sampler=sampler_gbc)
study_gbc.optimize(objective_gbc, n_trials=100)  # Ajustez n_trials selon les besoins

print("Number of finished trials: {}".format(len(study_gbc.trials)))
print("Best trial:")
trial = study_gbc.best_trial
print("Value: {}".format(trial.value))
print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
# Récupération des meilleurs hyperparamètres
best_params_gbc = study_gbc.best_params

# Création et entraînement du modèle avec les meilleurs hyperparamètres
best_gbc = GradientBoostingClassifier(**best_params_gbc, random_state=1)
best_gbc.fit(X_train_scaled, y_train)

# Prédiction et évaluation sur l'ensemble de validation
y_pred = best_gbc.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy on validation set: {accuracy:.4f}")


In [ ]:
best_params_gbc

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
# Création et entraînement du modèle avec les meilleurs hyperparamètres
model_full_gbc = GradientBoostingClassifier(**best_params_gbc, random_state=1)
model_full_gbc.fit(X_full_train_encoded, y_full_train)

# soumission

In [ ]:
# prédictions dans la base de données datest
y_test_pred = model_full_gbc.predict(X_test_encoded)
gbc_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_test_pred.flatten()})
gbc_model_soumission.dtypes

In [ ]:
gbc_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
gbc_model_soumission.to_csv(pa + "gbc_model_soumission.csv", index=False)

# adaboost

In [ ]:
import optuna
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Conversion des labels
#le = LabelEncoder()
#y_train_encoded = le.fit_transform(y_train)
#y_val_encoded = le.transform(y_val)

def objective_adaboost(trial):
    # Définition des hyperparamètres à optimiser
    n_estimators = trial.suggest_int("n_estimators", 10, 500)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
    
    # Utilisation de DecisionTreeClassifier comme estimateur de base
    base_max_depth = trial.suggest_int("base_max_depth", 1, 10)
    base_classifier = DecisionTreeClassifier(max_depth=base_max_depth)

    classifier = AdaBoostClassifier(
        base_estimator=base_classifier,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=42
    )
    
    classifier.fit(X_train_scaled, y_train)
    y_pred = classifier.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

sampler_adaboost = optuna.samplers.TPESampler(seed=42)
study_adaboost = optuna.create_study(direction="maximize", sampler=sampler_adaboost)
study_adaboost.optimize(objective_adaboost, n_trials=100)  # Ajustez n_trials selon les besoins

print("Number of finished trials: {}".format(len(study_adaboost.trials)))
print("Best trial:")
trial = study_adaboost.best_trial
print("Value: {}".format(trial.value))
print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
best_params_adaboost = study_adaboost.best_params

# Utiliser les meilleurs hyperparamètres 
best_base_classifier = DecisionTreeClassifier(max_depth=best_params_adaboost["base_max_depth"])
best_classifier = AdaBoostClassifier(
    base_estimator=best_base_classifier,
    n_estimators=best_params_adaboost["n_estimators"],
    learning_rate=best_params_adaboost["learning_rate"],
    random_state=42
)

best_classifier.fit(X_train_scaled, y_train)

y_pred = best_classifier.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy on validation set:", accuracy)


In [ ]:
best_params_adaboost  

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
best_base_classifier = DecisionTreeClassifier(max_depth=best_params_adaboost["base_max_depth"])
model_full_adaboost = AdaBoostClassifier(
    base_estimator=best_base_classifier,
    n_estimators=best_params_adaboost["n_estimators"],
    learning_rate=best_params_adaboost["learning_rate"],
    random_state=42
)

model_full_adaboost.fit(X_full_train_encoded, y_full_train)

# soumission

In [ ]:
# prédictions dans la base de données datest
y_test_pred = model_full_adaboost.predict(X_test_encoded)
adaboost_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_test_pred.flatten()})
adaboost_model_soumission.dtypes

In [ ]:
adaboost_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
adaboost_model_soumission.to_csv(pa + "adaboost_model_soumission.csv", index=False)

# réseau de neurones

In [ ]:
import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

def objective(trial):
    # Définir les hyperparamètres à optimiser
    n_layers = trial.suggest_int('n_layers', 1, 3)
    n_neurons = trial.suggest_int('n_neurons', 1, 16)
    hidden_layer_sizes = (n_neurons,) * n_layers
    alpha = trial.suggest_float('alpha', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 8, 32)
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-6, 1e-3, log=True)

    # Créer et entraîner le classificateur
    rn_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                        activation='relu',
                        solver='adam',
                        learning_rate='constant',
                        alpha=alpha,
                        max_iter=1000,  # réduit pour un essai rapide
                        random_state=1,
                        verbose=1,
                        tol=0.0001,
                        batch_size=batch_size,
                        n_iter_no_change=20,  # arrêt plus rapide si pas d'amélioration
                        learning_rate_init=learning_rate_init)
    rn_model.fit(X_train_scaled, y_train)

    # Évaluer le classificateur
    y_pred = rn_model.predict(X_val_scaled)
    return accuracy_score(y_val, y_pred)

sampler = optuna.samplers.TPESampler(seed=1)
study_rn = optuna.create_study(direction="maximize", sampler=sampler)
study_rn.optimize(objective, n_trials=200)  # réduit pour un essai rapide

print("Number of finished trials: {}".format(len(study_rn.trials)))
print("Best trial:")
trial = study_rn.best_trial
print("  Value (accuracy): {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
rn_best_params = study_rn.best_params

best_n_layers = rn_best_params['n_layers']
best_n_neurons = rn_best_params['n_neurons']
best_hidden_layer_sizes = (best_n_neurons,) * best_n_layers
best_alpha = rn_best_params['alpha']
best_batch_size = rn_best_params['batch_size']
best_learning_rate_init = rn_best_params['learning_rate_init']

rn_best = MLPClassifier(
    hidden_layer_sizes=best_hidden_layer_sizes,
    activation='relu',
    solver='adam',
    learning_rate='constant',
    alpha=best_alpha,
    max_iter=1000,  # 1000 pour un entraînement complet
    random_state=1,
    verbose=1,
    tol=0.0001,
    batch_size=best_batch_size,
    n_iter_no_change=20,
    learning_rate_init=best_learning_rate_init
)


rn_best.fit(X_train_scaled, y_train)


y_pred_val = rn_best.predict(X_val_scaled)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f"Accuracy on validation set: {accuracy_val:.4f}")


In [ ]:
rn_best_params

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
#X_full_train_encoded = ct.fit_transform(X_full_train)

# Normaliser les données en utilisant StandardScaler
scaler = StandardScaler()
X_full_train_encoded_scaled = scaler.fit_transform(X_full_train_encoded)

In [ ]:





full_rn_best = MLPClassifier(
    hidden_layer_sizes=best_hidden_layer_sizes,
    activation='relu',
    solver='adam',
    learning_rate='constant',
    alpha=best_alpha,
    max_iter=1000,  # 1000 pour un entraînement complet
    random_state=1,
    verbose=1,
    tol=0.0001,
    batch_size=best_batch_size,
    n_iter_no_change=20,
    learning_rate_init=best_learning_rate_init
)


full_rn_best.fit(X_full_train_encoded_scaled, y_full_train)

# soumission

In [ ]:
# prédictions dans la base de données datest

X_test_encoded_scaled = scaler.transform(X_test_encoded)

y_pred_test = full_rn_best.predict(X_test_encoded_scaled)


rn_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_pred_test.flatten()})
rn_model_soumission.dtypes

In [ ]:
rn_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
rn_model_soumission.to_csv(pa + "rn_model_soumission.csv", index=False)

# STACKING

In [ ]:
# staking 2 modeles avec 100 trials ## el mejor***************************************************
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier

def objective_stacking(trial):
    # Suggérer des hyperparamètres pour le méta-modèle
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
    n_estimators = trial.suggest_int("n_estimators", 10, 300)
    max_depth = trial.suggest_int("max_depth", 2, 10)

    # Créer le méta-modèle avec les hyperparamètres suggérés
    meta_model = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    
    # Noter que les modèles de base sont déjà optimisés
    estimators = [('model_catboost', model), ('rn_best', rn_best)]
    
    stacking_1 = StackingClassifier(estimators=estimators, final_estimator=meta_model)
    
    stacking_1.fit(X_train_scaled, y_train)
    y_pred = stacking_1.predict(X_val_scaled)
    accuracy = accuracy_score(y_val, y_pred)
    
    return accuracy

sampler_stacking = TPESampler(seed=1)
study_stacking_1 = optuna.create_study(direction="maximize", sampler=sampler_stacking)
study_stacking_1.optimize(objective_stacking, n_trials=100)  # Augmenter le nombre d'essais si nécessaire

print("Number of finished trials: {}".format(len(study_stacking_1.trials)))
print("Best trial:")
trial_stacking = study_stacking_1.best_trial
print("  Value (accuracy): {}".format(trial_stacking.value))
print("  Params: ")
for key, value in trial_stacking.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
# 1. Récupérer les meilleurs hyperparamètres de notre étude Optuna
stk1_best_params = study_stacking_1.best_params

learning_rate_best = stk1_best_params["learning_rate"]
n_estimators_best = stk1_best_params["n_estimators"]
max_depth_best = stk1_best_params["max_depth"]

# 2. Entraîner le modèle de stacking avec ces hyperparamètres
meta_model_best_1 = GradientBoostingClassifier(learning_rate=learning_rate_best, 
                                             n_estimators=n_estimators_best, 
                                             max_depth=max_depth_best)

estimators_best_1 = [('model_catboost', model), ('rn_best', rn_best)]

stacking_best_1 = StackingClassifier(estimators=estimators_best_1, final_estimator=meta_model_best_1)
stacking_best_1.fit(X_train_scaled, y_train)

# 3. Valider sur l'ensemble de validation
y_pred_val = stacking_best_1.predict(X_val_scaled)
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Accuracy on validation set:", accuracy_val)


In [ ]:
stk1_best_params

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
full_stacking_best_1 = StackingClassifier(estimators=estimators_best_1, final_estimator=meta_model_best_1)
full_stacking_best_1.fit(X_full_train_encoded_scaled, y_full_train)


# soumission

In [ ]:
# prédictions dans la base de données datest

y_pred_test_stacking_1 = full_stacking_best_1.predict(X_test_encoded_scaled)


stacking_1_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_pred_test_stacking_1.flatten()})
stacking_1_model_soumission.dtypes

In [ ]:
stacking_1_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
stacking_1_model_soumission.to_csv(pa + "stacking_1_model_soumission.csv", index=False)

# importance des variables

In [ ]:
transformed_feature_names = ct.get_feature_names_out()
transformed_feature_names


import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(stacking_best_1, random_state=1).fit(X_val_scaled, y_val)
display(eli5.show_weights(perm,top=None, feature_names = transformed_feature_names))

# réseau de neurones bayésien

In [ ]:
#import torch

#Convertir les données en tenseurs PyTorch :
#X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
#y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
#X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
#y_val_tensor = torch.tensor(y_val, dtype=torch.int64)



#y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
#y_train_tensor
from sklearn.preprocessing import LabelEncoder

# Création d'un objet LabelEncoder
label_encoder = LabelEncoder()

# Adapter le LabelEncoder aux données et transformer les étiquettes pour commencer à 0
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# optimisation reseaux  bayésien  avec optuna

In [ ]:
#Définition du Modèle
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

def create_bayesian_network(n_layers, n_neurons, in_features, out_features):
    layers = []
    layers.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=in_features, out_features=n_neurons))
    layers.append(nn.ReLU())
    for _ in range(n_layers - 1):
        layers.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=n_neurons, out_features=n_neurons))
        layers.append(nn.ReLU())
    layers.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=n_neurons, out_features=out_features))
    return nn.Sequential(*layers)


from sklearn.base import BaseEstimator, ClassifierMixin


import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
from sklearn.base import BaseEstimator, ClassifierMixin
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

class BayesianNNWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model, n_epochs, batch_size, learning_rate_init, kl_weight, alpha, tol, max_epochs_no_improve):
        self.model = model
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.learning_rate_init = learning_rate_init
        self.kl_weight = kl_weight
        self.alpha = alpha
        self.tol = tol
        self.max_epochs_no_improve = max_epochs_no_improve
        
        
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate_init, weight_decay=self.alpha)

    def fit(self, X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        if isinstance(y, pd.Series):
            y = y.to_numpy()  # 
        y_tensor = torch.tensor(y, dtype=torch.int64)
        
        train_dataset = TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=self.batch_size, shuffle=True)
        
        best_loss = float('inf')
        epochs_no_improve = 0


        for epoch in range(self.n_epochs):
            total_loss = 0
            for batch_X, batch_y in train_loader:
                pre = self.model(batch_X)
                ce = self.ce_loss(pre, batch_y )
                kl = self.kl_loss(self.model)
                cost = ce + self.kl_weight * kl
                self.optimizer.zero_grad()
                cost.backward()
                self.optimizer.step()
                total_loss += cost.item()

            average_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{self.n_epochs}, Average Loss: {average_loss:.4f}")
            
                # Logique d'arrêt anticipé
            if average_loss < best_loss - self.tol:
                best_loss = average_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= self.max_epochs_no_improve:
                print(f"Training loss did not improve more than tol={self.tol} for {self.max_epochs_no_improve} consecutive epochs. Stopping.")
                break

        return self

    def predict(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            y_pred_tensor = self.model(X_tensor)
            _, predicted = torch.max(y_pred_tensor.data, 1)
            #predicted += 1
            return predicted.cpu().numpy()




In [ ]:
# Fixer les graines
#import random
#np.random.seed(1)
#random.seed(1)
#torch.manual_seed(1)

# Assurer un comportement déterministe dans PyTorch (important si vous utilisez un GPU)
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False


def objective(trial, X_train_scaled, y_train, X_val_scaled, y_val, n_epochs=1000):
    # Suggérer des hyperparamètres
    n_layers = trial.suggest_int('n_layers', 4, 6)
    n_neurons = trial.suggest_int('n_neurons', 1, 16)
    alpha = trial.suggest_float('alpha', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    learning_rate_init = trial.suggest_float("learning_rate_init", 1e-6, 1e-3, log=True)
    kl_weight = trial.suggest_float("kl_weight", 0.01, 0.1, log=True)

    model_bnn = create_bayesian_network(n_layers, n_neurons, in_features=X_train_scaled.shape[1], out_features=len(np.unique(y_train_encoded)))
    bayesian_nn = BayesianNNWrapper(model_bnn, n_epochs, batch_size, learning_rate_init, kl_weight, alpha, tol=0.0001, max_epochs_no_improve=30)

    # Entraîner le modèle avec les données d'origine
    bayesian_nn.fit(X_train_scaled, y_train_encoded)

    # Évaluer le modèle
    y_pred = bayesian_nn.predict(X_val_scaled)
    accuracy = accuracy_score(y_val_encoded, y_pred)

    return accuracy

# Configuration d'Optuna et lancement de l'optimisation
sampler = optuna.samplers.TPESampler(seed=1)
study_bn = optuna.create_study(direction="maximize", sampler=sampler)
study_bn.optimize(lambda trial: objective(trial, X_train_scaled, y_train_encoded, X_val_scaled, y_val_encoded, n_epochs=1000), n_trials=200)

# Afficher les résultats
print("Number of finished trials: {}".format(len(study_bn.trials)))
print("Best trial:")
trial = study_bn.best_trial
print("  Value (accuracy): {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


In [ ]:
# Fixer les graines
#import random
#np.random.seed(1)
#random.seed(1)
#torch.manual_seed(1)

# Assurer un comportement déterministe dans PyTorch (important si vous utilisez un GPU)
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

# Récupérer les meilleurs hyperparamètres
bn_best_params = study_bn.best_params

# Créer le modèle bayésien avec les meilleurs hyperparamètres
bayesian_model = create_bayesian_network(n_layers=bn_best_params['n_layers'], 
                                         n_neurons=bn_best_params['n_neurons'], 
                                         in_features=X_train_scaled.shape[1], 
                                         out_features=len(np.unique(y_train_encoded)))

bayesian_nn = BayesianNNWrapper(
    model=bayesian_model,
    n_epochs=1000, 
    batch_size=bn_best_params['batch_size'],
    learning_rate_init=bn_best_params['learning_rate_init'],
    kl_weight=bn_best_params['kl_weight'],
    alpha=bn_best_params['alpha'],
    tol=0.0001, 
    max_epochs_no_improve=30
)
# Étape 2: Entraîner le modèle sur les données d'entraînement
bayesian_nn.fit(X_train_scaled, y_train_encoded)

# Étape 3: Évaluer le modèle sur les données de validation
y_pred = bayesian_nn.predict(X_val_scaled)
accuracy = accuracy_score(y_val_encoded, y_pred)
print("Accuracy on validation data:", accuracy)

In [ ]:
bn_best_params

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
#y_full_train = datrain['y']
#  datos completos X_full_train_encoded_scaled, y_full_train
# Création d'un objet LabelEncoder
label_encoder = LabelEncoder()

# Adapter le LabelEncoder aux données et transformer les étiquettes pour commencer à 0
y_full_train_encoded = label_encoder.fit_transform(y_full_train)

# Créer le modèle bayésien avec les meilleurs hyperparamètres
bayesian_model = create_bayesian_network(n_layers=bn_best_params['n_layers'], 
                                         n_neurons=bn_best_params['n_neurons'], 
                                         in_features=X_full_train_encoded_scaled.shape[1], 
                                         out_features=len(np.unique(y_train)))

full_bayesian_wrapper = BayesianNNWrapper(
    model=bayesian_model,
    n_epochs=1000, 
    batch_size=bn_best_params['batch_size'],
    learning_rate_init=bn_best_params['learning_rate_init'],
    kl_weight=bn_best_params['kl_weight'],
    alpha=bn_best_params['alpha'],
    tol=0.0001, 
    max_epochs_no_improve=30
)
# Étape 2: Entraîner le modèle sur les données d'entraînement
full_bayesian_wrapper.fit(X_full_train_encoded_scaled, y_full_train_encoded)

# soumission

In [ ]:
# prédictions dans la base de données datest

#X_test_encoded_scaled = scaler.transform(X_test_encoded)

y_pred_test = full_bayesian_wrapper.predict(X_test_encoded_scaled)
y_pred_original_test = label_encoder.inverse_transform(y_pred_test)

bnn_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_pred_original_test.flatten()})
bnn_model_soumission.dtypes

In [ ]:
np.unique(y_pred_test)

In [ ]:
np.unique(y_pred_original_test)

In [ ]:
bnn_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
bnn_model_soumission.to_csv(pa + "bnn_model_soumission.csv", index=False)

# STACKING 2

In [ ]:
# staking2 3 modeles avec 100 trials ## 
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier

def objective_stacking(trial):
    # Suggérer des hyperparamètres pour le méta-modèle
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.3, log=True)
    n_estimators = trial.suggest_int("n_estimators", 10, 300)
    max_depth = trial.suggest_int("max_depth", 2, 10)

    # Créer le méta-modèle avec les hyperparamètres suggérés
    meta_model_2 = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)
    
    # Ajout du modèle bayésien à la liste des modèles pour le stacking
    estimators_2 = [('model_catboost', model), ('rn_best', rn_best), ('bayesian_nn', bayesian_nn)]
    
    stacking_2 = StackingClassifier(estimators=estimators_2, final_estimator=meta_model_2)
    
    stacking_2.fit(X_train_scaled, y_train_encoded)
    y_pred = stacking_2.predict(X_val_scaled)
    accuracy = accuracy_score(y_val_encoded, y_pred)
    
    return accuracy

sampler_stacking = TPESampler(seed=1)
study_stacking_2 = optuna.create_study(direction="maximize", sampler=sampler_stacking)
study_stacking_2.optimize(objective_stacking, n_trials=100)  # Augmenter le nombre d'essais si nécessaire

print("Number of finished trials: {}".format(len(study_stacking_2.trials)))
print("Best trial:")
trial_stacking = study_stacking_2.best_trial
print("  Value (accuracy): {}".format(trial_stacking.value))
print("  Params: ")
for key, value in trial_stacking.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# 1. Récupérer les meilleurs hyperparamètres de notre étude Optuna
stk2_best_params = study_stacking_2.best_params

learning_rate_best = stk2_best_params["learning_rate"]
n_estimators_best = stk2_best_params["n_estimators"]
max_depth_best = stk2_best_params["max_depth"]

# 2. Entraîner le modèle de stacking avec ces hyperparamètres
meta_model_best_2 = GradientBoostingClassifier(learning_rate=learning_rate_best, 
                                             n_estimators=n_estimators_best, 
                                             max_depth=max_depth_best)

estimators_best_2 = [('model_catboost', model), ('rn_best', rn_best), ('bayesian_nn', bayesian_nn)]

stacking_best_2 = StackingClassifier(estimators=estimators_best_2, final_estimator=meta_model_best_2)
stacking_best_2.fit(X_train_scaled, y_train_encoded)

# 3. Valider sur l'ensemble de validation
y_pred_val = stacking_best_2.predict(X_val_scaled)
accuracy_val = accuracy_score(y_val_encoded, y_pred_val)
print("Accuracy on validation set:", accuracy_val)


In [ ]:
stk2_best_params

# entraînement données complètes datrain pour prédire avec datest

In [ ]:
full_stacking_best_2 = StackingClassifier(estimators=estimators_best_2, final_estimator=meta_model_best_2)
full_stacking_best_2.fit(X_full_train_encoded_scaled, y_full_train_encoded)


# soumission

In [ ]:
# prédictions dans la base de données datest

y_pred_test_stacking_2 = full_stacking_best_2.predict(X_test_encoded_scaled)
y_pred_original_test_stacking_2 = label_encoder.inverse_transform(y_pred_test_stacking_2)

stacking_2_model_soumission = pd.DataFrame({'id': datest['id'], 'y': y_pred_original_test_stacking_2.flatten()})
stacking_2_model_soumission.dtypes

In [ ]:
stacking_2_model_soumission

In [ ]:
pa = "C:/Users/a/Desktop/MAITRISE HEC/SESSION_4_AUTOMNE_2023/DATA MINING/documents_etudiants_A2023/documents_etudiants_A2023/travail_equipe_fichiers_etudiants/remise_automne_2023/soumission_kaggle/"
stacking_2_model_soumission.to_csv(pa + "stacking_2_model_soumission.csv", index=False)